In [ ]:
import pandas as pd
from os.path import join
from os import mkdir
import torch as th
from torch.nn import functional as F
from tqdm import tqdm
from liquid_networks.data.transform import standardize_column

In [ ]:
dataset_path = "/home/samuel/Téléchargements/hms-harmful-brain-activity-classification"
output_folder = "/home/samuel/Téléchargements/hms-harmful-brain-activity-classification/extracted_eeg_data"

mkdir(output_folder)

In [ ]:
train_df = pd.read_csv(join(dataset_path, "train.csv"), sep=",")

In [ ]:
target_cols = [
    "seizure_vote",
    "lpd_vote",
    "gpd_vote",
    "lrda_vote",
    "grda_vote",
    "other_vote",
]

sample_rate_eegs = 200
nb_seconds_eegs = 50
sample_rate_spec = 0.5
nb_seconds_spectrogram = 10 * 60

In [ ]:
last_eeg_id = None
last_eeg = None

for i, row in tqdm(list(train_df.iterrows())):
    # TODO read each parquet once
    #spec = pd.read_parquet(
    #    join(dataset_path, "train_spectrograms", f"{row['spectrogram_id']}.parquet"),
    #    engine="pyarrow",
    #)
        
    if row["eeg_id"] != last_eeg_id:
        last_eeg = pd.read_parquet(
            join(dataset_path, "train_eegs", f"{row['eeg_id']}.parquet"),
            engine="pyarrow",
        )
        last_eeg_id = row['eeg_id']

    #sub_spec = spec.iloc[
    #    int(sample_rate_spec * row["spectrogram_label_offset_seconds"])
    #    : int(sample_rate_spec * (row["spectrogram_label_offset_seconds"] + nb_seconds_spectrogram))
    #].drop("time", axis=1)
    
    sub_eeg = last_eeg.iloc[
        int(sample_rate_eegs * row["eeg_label_offset_seconds"])
        : int(sample_rate_eegs * (row["eeg_label_offset_seconds"] + nb_seconds_eegs))
    ]
    
    targets_df = row[target_cols].astype(float).fillna(0.0)
    classes = targets_df / targets_df.sum()
    
    # specs_t = th.nan_to_num(th.tensor(sub_spec.to_numpy().T, dtype=th.float))
    eeg_t = th.nan_to_num(th.tensor(sub_eeg.to_numpy().T, dtype=th.float))
    classes_t = th.nan_to_num(th.tensor(classes.to_numpy(), dtype=th.float))
    
    # specs_t = F.pad(specs_t, (specs_t.size(1) - int(nb_seconds_spectrogram * sample_rate_spec), 0))
    eeg_t = F.pad(eeg_t, (eeg_t.size(1) - int(nb_seconds_eegs * sample_rate_eegs), 0))

    # th.save(specs_t, join(output_folder, f"{i}_spec.pt"))
    th.save(eeg_t, join(output_folder, f"{i}_eeg.pt"))
    th.save(classes_t, join(output_folder, f"{i}_classes.pt"))